<a href="https://colab.research.google.com/github/LorenzoZaccagnini/cryptography-works/blob/master/AES_plain_text_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: [Lorenzo Zaccagnini](https://www.linkedin.com/in/lorenzo-zaccagnini/)

# Crack AES using IV attack

In this notebook I'll try to break an AES symmetric encryption. According to the Snowden documents, the NSA is doing research on whether a cryptographic attack based on tau statistic may help to break AES.[28]

At present, there is no known practical attack that would allow someone without knowledge of the key to read data encrypted by AES when correctly implemented.

[Source Wikipedia](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard)

The actual problem lies in reusing the same initialization vector value (IV) with multiple encryptions with the same key. The IV values should be nonces (or 'number used only once') to protect against this attack. Usually these nonce values are achieved by using a running counter value added to the original IV value (IV || CTR[i]), hence the name counter mode. Let me demonstrate the attack and how to prevent it:

Ciphertext1 = Plaintext1 ⊕ AES(key, IV)
Ciphertext2 = Plaintext2 ⊕ AES(key, IV)


Which leads to the following ciphertext pair:
Ciphertext1 ⊕ Ciphertext2  = Plaintext1 ⊕ AES(key, IV) ⊕  Plaintext2 ⊕ AES(key, IV)


Now, because the (key, IV) pair is reused, the AES(key, IV) will yield the same result for both ciphertexts. 
This means that an attacker can now compute Ciphertext pairs easily by cancelling the AES encryption out of the equation (XORing anything by itself will always yield to 0):
Ciphertext1 ⊕ Ciphertext2  = Plaintext1 ⊕  Plaintext2 

Therefore an attacker can easily get the Plaintext2 value by computing the following operation:
Plaintext2 = Plaintext1 ⊕ Ciphertext1 ⊕ Ciphertext2

Source [John Hammond - Arttu Paju comment](https://www.youtube.com/watch?v=Gtfr1dBGzHg) 

**The plaintext is:**
No right of private conversation was enumerated in the Constitution. I don't suppose it occurred to anyone at the time that it could be prevented

Now we download AES encrypted outputs



In [5]:
!wget https://raw.githubusercontent.com/LorenzoZaccagnini/cryptography-works/master/AES%20plaintext%20attack/output.txt

--2021-06-10 15:43:43--  https://raw.githubusercontent.com/LorenzoZaccagnini/cryptography-works/master/AES%20plaintext%20attack/output.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340 [text/plain]
Saving to: ‘output.txt’

output.txt          100%[===================>]     340  --.-KB/s    in 0s      

2021-06-10 15:43:43 (15.7 MB/s) - ‘output.txt’ saved [340/340]



In [6]:
cat output.txt

464851522838603926f4422a4ca6d81b02f351b454e6f968a324fcc77da30cf979eec57c8675de3bb92f6c21730607066226780a8d4539fcf67f9f5589d150a6c7867140b5a63de2971dc209f480c270882194f288167ed910b64cf627ea6392456fa1b648afd0b239b59652baedc595d4f87634cf7ec4262f8c9581d7f56dc6f836cfe696518ce434ef4616431d4d1b361c
4b6f25623a2d3b3833a8405557e7e83257d360a054c2ea


**we have the two ciphertexts, and we have one of the plaintexts**

# Some math
C1​=P1​⊕K
C2​=P2​⊕K

**Then**

If we now XOR the two ciphertexts together, we get

C1​⊕C2​=P1​⊕K⊕P2​⊕K

Because an XOR is its own inverse

C1​⊕C2​=P1​⊕P2

If we know one of the plaintexts, we can XOR the value we have from above with that plaintext, and get the other

C1⊕C2⊕P1=P1⊕P2⊕P1=P2

In [20]:
ciphertext1 = '464851522838603926f4422a4ca6d81b02f351b454e6f968a324fcc77da30cf979eec57c8675de3bb92f6c21730607066226780a8d4539fcf67f9f5589d150a6c7867140b5a63de2971dc209f480c270882194f288167ed910b64cf627ea6392456fa1b648afd0b239b59652baedc595d4f87634cf7ec4262f8c9581d7f56dc6f836cfe696518ce434ef4616431d4d1b361c'

ciphertext2 = '4b6f25623a2d3b3833a8405557e7e83257d360a054c2ea'

plaintext = b'No right of private conversation was enumerated in the Constitution. I don\'t suppose it occurred to anyone at the time that it could be prevented.'.hex()

def xor(hex1, hex2, getAscii = False):
  result = []

  for ind in range(0, len(hex1), 2):
    longIndex = ind
    shortIndex = ind%len(hex2)
    hexChar1 = hex1[longIndex:longIndex+2]
    byte1 = int(hexChar1, 16)

    hexChar2 = hex2[shortIndex:shortIndex+2]
    byte2 = int(hexChar2, 16)

    asciiNum = byte1 ^ byte2
    result.append(chr(asciiNum))


  out = ''.join(result)
  if getAscii:
    print('Result:', out)
    return out
  else:
    out = out.encode('utf-8').hex()
    print('Result:', out)
    return out
    

xored = xor(ciphertext2, ciphertext1)

flag = xor(xored, plaintext, True)

Result: 0d27743012155b01155c027f1b41302955203114002413
Result: CHTB{r3u53d_k3Y_4TT4cK}
